In [2]:
import rvnewop as rv
from glob import glob
import matplotlib.pyplot as plt
from itertools import accumulate

In [4]:
# remove st.hst, edn.hst, minver.hst, cubic.hst, wikisort.hst, nbody.hst
floating_point_hsts = list(map(lambda x: "./embench_hst/"+x, ["st.hst", "edn.hst", "minver.hst", "cubic.hst", "wikisort.hst", "nbody.hst"]))
for FILENAME in [x for x in glob("./embench_hst/*.hst") if x not in floating_point_hsts]:
    #print("Processing {}...".format(FILENAME))

    prog = rv.Histogram.parse(FILENAME, isa="32ICM")
    prog.findBasicBlocks()

    prog.addLivenessValuesToGraph()

    # total cycles = sum of all the frequencies
    total_cycles = sum(
        [inst.freq for bb in prog.basicBlocks for inst in bb.bbInstructions()]
    )

    new_instructions = []

    for i, b in enumerate((prog.getSubBlocks())):
        graph = b.constructDAG()

        # temp will store every permutation of candidate subgraphs
        temp = []
        for n in rv.analysis.findCandidateSubgraphs(prog, graph):
            subtree = rv.analysis.createSubtreeFromNode(graph, n)
            root = n

            temp.append(rv.Subgraph(subtree, root))

        # add candidate subgraph with highest score to new instructions list
        current_nodes = set()
        temp = sorted(temp, key=lambda sg: sg.score, reverse=True)
        while temp:
            if any([(n in current_nodes) for n in temp[0].graph.nodes]):
                temp.pop(0)
            else:
                new_instructions.append(temp.pop(0))

    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    inst_dict = {}
    for subgraph in new_instructions:
        key = rv.analysis.graphToParenString(subgraph.graph)

        if key in inst_dict:
            inst_dict[key].score += subgraph.score
        else:
            inst_dict[key] = subgraph

    new_instructions = [inst_dict[key] for key in inst_dict]

    max_depth = 0
    max_mult_depth = 0
    for sg in new_instructions:
        if sg.containsMultiplyInstruction():
            max_mult_depth = max(max_mult_depth, sg.depth)
        else:
            max_depth = max(max_depth, sg.depth)

    saved_cycles = sum([sg.score for sg in new_instructions])
    percent_cycles = float(saved_cycles) / total_cycles * 100
    #plt.clf()
    #plt.plot([x for x in range(len(new_instructions))],list(map(lambda x:x/total_cycles, accumulate([sg.score for sg in new_instructions]))))
    #plt.show()
    #print(
    #    """Saved {} cycles out of {} ({:.4}%) with {} new instructions""".format(
    #        saved_cycles, total_cycles, percent_cycles, len(new_instructions)
    #    )
    #)
    #print("max depth: {}\nmax mult depth: {}".format(max_depth, max_mult_depth))
    
    print("{},{:.4},{:.4},{:.4},{:.4},{:.4}".format(FILENAME[len("./embench_hst/"):-4], *[float(x.score) / saved_cycles for x in new_instructions[:5]]))


aha-mont64,0.141,0.1508,0.1418,0.4096,0.04698
ud,0.1029,0.2601,0.07719,0.1734,0.01425
slre,0.445,0.1425,0.03196,0.05151,0.02787
qrduino,0.1205,0.04474,0.03728,0.0904,0.07979
matmult-int,0.4152,0.3321,0.2491,0.0008474,0.000349
picojpeg,0.03523,0.03631,0.05737,0.04918,0.04098
sglib-combined,0.05549,0.249,0.03329,0.2199,0.01142
nsichneu,0.3987,0.1595,0.07975,0.07975,0.05354
coremark,0.2631,0.05225,0.05037,0.08261,0.08438
nettle-sha256,0.01298,0.006627,0.006492,0.01343,0.1558
huffbench,0.215,0.08409,0.3933,0.05579,0.05208
crc32,0.2093,0.1831,0.1308,0.1047,0.07849
nettle-aes,0.4752,0.2376,0.1288,0.007425,0.0297
statemate,0.3318,0.1327,0.08401,0.06635,0.06635
